In [1]:
import cv2
import numpy as np
import pandas as pd
from dataset.dataset import ImageDataset
from easydict import EasyDict as edict
import json
import pickle as pkl
from tqdm.notebook import tqdm
from Classification.model.classifier import Classifier

with open("config/example_EXP.json", "r") as f:
   cfg = edict(json.load(f))

In [27]:
train_data = pd.read_csv("D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/train_subsample.csv")
generated_data = pd.read_csv(r"D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/generated.csv")
# df = pd.concat([generated_data, train_data])

In [28]:
train_data.shape, generated_data.shape

((44683, 19), (5350, 19))

In [29]:
df = pd.concat([generated_data, train_data])

In [30]:
gfy2 = pd.DataFrame(df[['Cardiomegaly',
        'Edema',
        'Consolidation',
        'Atelectasis',
        'Pleural Effusion']].fillna(0))
for x in ['Cardiomegaly',
        'Edema',
        'Consolidation',
        'Atelectasis',
        'Pleural Effusion']:
    gfy2[x] = gfy2[x].apply(lambda t: max(t, 0))
gfy2

,Cardiomegaly,Edema,Consolidation,Atelectasis,Pleural Effusion
0,1.0,0.0,0.0,0.0,1.0
1,1.0,1.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...
44678,0.0,1.0,0.0,0.0,1.0
44679,0.0,0.0,0.0,0.0,0.0
44680,0.0,0.0,1.0,0.0,1.0
44681,0.0,1.0,1.0,0.0,1.0


In [32]:
gfy2.sum(axis=0) / len(gfy2)

1.096916035416625

In [26]:
for_valid = df.sample(frac=0.2)

In [62]:
gfy = pd.DataFrame(for_valid[['Cardiomegaly',
        'Edema',
        'Consolidation',
        'Atelectasis',
        'Pleural Effusion']].fillna(0))
for x in ['Cardiomegaly',
        'Edema',
        'Consolidation',
        'Atelectasis',
        'Pleural Effusion']:
    gfy[x] = gfy[x].apply(lambda t: max(t, 0))
gfy

,Cardiomegaly,Edema,Consolidation,Atelectasis,Pleural Effusion
18402,0.0,0.0,0.0,0.0,0.0
166511,0.0,0.0,0.0,0.0,1.0
146086,0.0,0.0,0.0,0.0,1.0
183009,0.0,0.0,0.0,0.0,1.0
61890,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...
94384,0.0,1.0,0.0,0.0,1.0
159565,0.0,0.0,0.0,0.0,0.0
179235,0.0,0.0,1.0,0.0,1.0
167361,0.0,1.0,1.0,0.0,1.0


In [74]:
gfy.sum(axis=0)

Cardiomegaly         5274.0
Edema               10454.0
Consolidation        2941.0
Atelectasis          6678.0
Pleural Effusion    17129.0
dtype: float64

In [65]:
gfy.sum(axis=0) / len(gfy)

Cardiomegaly        0.118031
Edema               0.233959
Consolidation       0.065819
Atelectasis         0.149453
Pleural Effusion    0.383345
dtype: float64

In [75]:
for_valid.to_csv("D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/train_subsample.csv", index=None)

In [20]:
for_train = df.merge(for_valid, how='outer', indicator=True).loc[lambda x : x['_merge']=='left_only']

In [22]:
for_train.to_csv("D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/generated_train.csv", index=None)
for_valid.to_csv("D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/generated_valid.csv", index=None)

In [ ]:
# CREATE_DATASET = False
#
# if CREATE_DATASET:
#     label_path = r"D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/valid.csv"
#     chexpert_dataset = ImageDataset(label_path, cfg, mode='test')
#     with open("dev/chexpert_dataset_test.pkl", "wb") as f:
#         pkl.dump(chexpert_dataset, f)
# else:
#     with open("dev/chexpert_dataset_test.pkl", "rb") as f:
#         chexpert_dataset = pkl.load(f)

In [2]:
# test_image = np.transpose(chexpert_dataset[0][0], (1, 2, 0))

In [3]:
# cv2.imshow('image', test_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [2]:
label_path = r"D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/generated.csv"
dataset_generated = ImageDataset(label_path, cfg, mode='train')

In [24]:
import pandas as pd

generated_data = pd.read_csv(r"D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/generated.csv")
train_data = pd.read_csv(r"D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/train_subsample.csv")

In [26]:
augmented_data = pd.concat([generated_data, train_data])
augmented_data.shape

(50033, 19)

In [78]:
augmented_data.to_csv(r"D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/augmented_subsample.csv", index=None, header=True)

In [79]:
label_path = r"D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/augmented_subsample.csv"
dataset_generated_test = ImageDataset(label_path, cfg, mode='train')

In [29]:
dataset = ImageDataset("D:/chexpert/chexpertchestxrays-u20210408/CheXpert-v1.0/train.csv", cfg, mode='train')

In [41]:
from torch.utils.data import Subset
import random

s = Subset(dataset, random.choices(range(len(dataset)), k=int(0.2 * len(dataset))))

In [42]:
next(iter(s))

(array([[[-2.      , -2.      , -1.984375, ..., -1.984375, -2.      ,
          -2.      ],
         [-2.      , -1.984375, -1.96875 , ..., -1.96875 , -1.984375,
          -2.      ],
         [-1.96875 , -1.953125, -1.75    , ..., -1.953125, -1.96875 ,
          -1.984375],
         ...,
         [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
           0.      ],
         [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
           0.      ],
         [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
           0.      ]],
 
        [[-2.      , -2.      , -1.984375, ..., -1.984375, -2.      ,
          -2.      ],
         [-2.      , -1.984375, -1.96875 , ..., -1.96875 , -1.984375,
          -2.      ],
         [-1.96875 , -1.953125, -1.75    , ..., -1.953125, -1.96875 ,
          -1.984375],
         ...,
         [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
           0.      ],
         [ 0.      ,  0.      ,  0.      , ...,  